In [ ]:
!pip install tensorflow matplotlib
import os
import zipfile
import numpy as np
import tensorflow as tf
!pip install --upgrade tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers



In [ ]:
import os
import zipfile
import tensorflow as tf

# Assuming the zip file is located in your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to the zip file
zip_path = '/content/drive/MyDrive/RemoteInternship ML/ML TASK#5/food.zip'

# Unzipping the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/food-11-dataset')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set dataset paths
train_dir = '/content/food-11-dataset/food11/train'
test_dir = '/content/food-11-dataset/food11/test'

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and augment data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 9900 images belonging to 11 classes.
Found 1100 images belonging to 11 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers

# Load the VGG16 model
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional base
conv_base.trainable = False

# Build the model
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(11, activation='softmax'))  # 11 food categories

# Compile the model
model.compile(optimizer=optimizers.RMSprop(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


309/309 ━━━━━━━━━━━━━━━━━━━━ 123s 388ms/step - accuracy: 0.2162 - loss: 2.2516 - val_accuracy: 0.4228 - val_loss: 1.7840
Epoch 2/30
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 712us/step - accuracy: 0.3750 - loss: 1.9177 - val_accuracy: 0.3333 - val_loss: 1.6868
Epoch 3/30


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


309/309 ━━━━━━━━━━━━━━━━━━━━ 121s 385ms/step - accuracy: 0.4128 - loss: 1.7862 - val_accuracy: 0.4936 - val_loss: 1.5598
Epoch 4/30
309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 638us/step - accuracy: 0.3125 - loss: 1.8955 - val_accuracy: 0.4167 - val_loss: 2.1744
Epoch 5/30
309/309 ━━━━━━━━━━━━━━━━━━━━ 121s 385ms/step - accuracy: 0.4686 - loss: 1.6259 - val_accuracy: 0.5450 - val_loss: 1.4415
Epoch 6/30
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 610us/step - accuracy: 0.4375 - loss: 1.6770 - val_accuracy: 0.5833 - val_loss: 1.4510
Epoch 7/30
309/309 ━━━━━━━━━━━━━━━━━━━━ 121s 387ms/step - accuracy: 0.4894 - loss: 1.5570 - val_accuracy: 0.5607 - val_loss: 1.3904
Epoch 8/30
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 627us/step - accuracy: 0.6250 - loss: 1.4625 - val_accuracy: 0.6667 - val_loss: 1.1353
Epoch 9/30
309/309 ━━━━━━━━━━━━━━━━━━━━ 121s 387ms/step - accuracy: 0.5269 - loss: 1.4600 - val_accuracy: 0.5524 - val_loss: 1.3385
Epoch 10/30
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 679us/step - accuracy: 0.5938 - loss: 1.2636 - va

In [ ]:
# Food-calorie mapping
food_calorie_map = {
    'applepie': 237,
    'cheesecake': 321,
    'chickencurry': 220,
    'frenchfries': 312,
    'friedrice': 163,
    'hamburger': 295,
    'hotdog': 150,
    'icecream': 207,
    'omelette': 154,
    'pizza': 266,
    'sushi': 130,
}

def estimate_calories(predicted_class):
    food_item = list(train_generator.class_indices.keys())[predicted_class]
    return food_calorie_map[food_item]


In [ ]:
def predict_and_estimate_calories(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)

    food_item = list(train_generator.class_indices.keys())[predicted_class]
    calories = estimate_calories(predicted_class)

    return food_item, calories

# Test the function with a sample image
sample_image_path = '/content/food-11-dataset/food11/test/cheesecake/106769.jpg'
# Replace with a path to an actual image
food_item, calories = predict_and_estimate_calories(sample_image_path)
print(f'Food Item: {food_item}, Estimated Calories: {calories}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Food Item: cheesecake, Estimated Calories: 321
